# CS105 Team 2 Final Project

## Proposal

We want to study the University UCI database. We want to answer questions such as if university expenses predicts quality of life, if sat-score predicts percent-admittance, if the data can be clustered, etc.

## Data Cleaning

The University dataset is quite old and takes some effort to turn it into a usable pandas dataframe. First, we've removed all commented-out features. These are missing features that will be replaced in the future. Second, we've removed all duplicate entries; some universities show up more than once with little difference in features, so we've removed those duplicate instances. Third, we've transformed the data into a comma-separated list. Finally, we can read this data into a pandas dataframe:

In [1]:
import pandas as pd

features = ['university-name', 'state', 'location', 'control', 'number-of-students', 'male-percentage',
            'student-percentage', 'sat-verbal', 'sat-math', 'expenses', 'percent-financial-aid',
            'number-of-applicants', 'percent-admittance', 'percent-enrolled', 'academics', 'social',
            'quality-of-life', 'academic-emphasis']
df = pd.read_csv('university.csv', 
                   names = features)

df.head(4)

,university-name,state,location,control,number-of-students,male-percentage,student-percentage,sat-verbal,sat-math,expenses,percent-financial-aid,number-of-applicants,percent-admittance,percent-enrolled,academics,social,quality-of-life,academic-emphasis
0,ADELPHI,NEWYORK,SUBURBAN,PRIVATE,5-10,30:70,15:1,500.0,475.0,7-10,60.0,4-7,70.0,40.0,2.0,2.0,2.0,"['BUSINESS-ADMINISTRATION', 'BIOLOGY']"
1,ARIZONA-STATE,ARIZONA,URBAN,STATE,20+,50:50,20:1,450.0,500.0,4-7,50.0,17+,80.0,60.0,3.0,4.0,5.0,"['BUSINESS-EDUCATION', 'ENGINEERING', 'ACCOUNT..."
2,BOSTON-COLLEGE,MASSACHUSETTS,SUBURBAN,PRIVATE,5-10,40:60,20:1,500.0,550.0,10+,60.0,10-13,50.0,40.0,4.0,5.0,3.0,"['ECONOMICS', 'BIOLOGY', 'ENGLISH']"
3,BOSTON-UNIVERSITY,MASSACHUSETTS,URBAN,PRIVATE,10-15,45:55,12:1,550.0,575.0,10+,60.0,13-17,60.0,40.0,4.0,4.0,3.0,"['BUSINESS-ADMINISTRATION', 'PSYCHOLOGY', 'LIB..."


There are still some issues with the data. We have some missing features, some numbers are represented as a range, and ratios should be floats. We'll convert all numerical data into floats. For features with ranges, we'll use the convention of taking the lower number within the range for all values.

In [2]:
def convertRatio(x):
    if isinstance(x, float) or x == 'NaN':
        return x
    a, b = x.split(':')
    c = float(a)/(float(a) + float(b))
    return c

def getLower(x):
    if isinstance(x, float) or x == 'NaN':
        return x
    start_index = 0
    while not x[start_index].isnumeric():
        start_index += 1
    end_index = start_index
    while end_index < len(x) and x[end_index].isnumeric():
        end_index += 1
    return float(x[start_index:end_index]) * 1000.0

def convertPercentage(x):
    if isinstance(x, float) and x <= 1.0 or x == 'NaN':
        return x
    return float(x)/100

df['number-of-students'] = df['number-of-students'].apply(getLower)
df['male-percentage'] = df['male-percentage'].apply(convertRatio)
df['student-percentage'] = df['student-percentage'].apply(convertRatio)
df['sat-verbal'] = df['sat-verbal'].astype(float)
df['sat-math'] = df['sat-math'].astype(float)
df['expenses'] = df['expenses'].apply(getLower)
df['percent-financial-aid'] = df['percent-financial-aid'].apply(convertPercentage)
df['number-of-applicants'] = df['number-of-applicants'].apply(getLower)
df['percent-admittance'] = df['percent-admittance'].apply(convertPercentage)
df['percent-enrolled'] = df['percent-enrolled'].apply(convertPercentage)
df['academics'] = df['academics'].astype(float)
df['social'] = df['social'].astype(float)
df['quality-of-life'] = df['quality-of-life'].astype(float)

df.head(4)

,university-name,state,location,control,number-of-students,male-percentage,student-percentage,sat-verbal,sat-math,expenses,percent-financial-aid,number-of-applicants,percent-admittance,percent-enrolled,academics,social,quality-of-life,academic-emphasis
0,ADELPHI,NEWYORK,SUBURBAN,PRIVATE,5000.0,0.30,0.937500,500.0,475.0,7000.0,0.6,4000.0,0.7,0.4,2.0,2.0,2.0,"['BUSINESS-ADMINISTRATION', 'BIOLOGY']"
1,ARIZONA-STATE,ARIZONA,URBAN,STATE,20000.0,0.50,0.952381,450.0,500.0,4000.0,0.5,17000.0,0.8,0.6,3.0,4.0,5.0,"['BUSINESS-EDUCATION', 'ENGINEERING', 'ACCOUNT..."
2,BOSTON-COLLEGE,MASSACHUSETTS,SUBURBAN,PRIVATE,5000.0,0.40,0.952381,500.0,550.0,10000.0,0.6,10000.0,0.5,0.4,4.0,5.0,3.0,"['ECONOMICS', 'BIOLOGY', 'ENGLISH']"
3,BOSTON-UNIVERSITY,MASSACHUSETTS,URBAN,PRIVATE,10000.0,0.45,0.923077,550.0,575.0,10000.0,0.6,13000.0,0.6,0.4,4.0,4.0,3.0,"['BUSINESS-ADMINISTRATION', 'PSYCHOLOGY', 'LIB..."


We have some missing features in the 'student-percentage', 'sat-verbal', 'sat-math', 'percent-financial-aid', 'number-of-applicants', 'percent-admittance', 'percent-enrolled', 'academics', 'social', and 'quality-of-life' columns. For all of these, we'll take the median value across all rows and replace the missing value with it.

In [3]:
df['student-percentage'] = df['student-percentage'].fillna(df['student-percentage'].median())
df['sat-verbal'] = df['sat-verbal'].fillna(df['sat-verbal'].median())
df['sat-math'] = df['sat-math'].fillna(df['sat-math'].median())
df['percent-financial-aid'] = df['percent-financial-aid'].fillna(df['percent-financial-aid'].median())
df['number-of-applicants'] = df['number-of-applicants'].fillna(df['number-of-applicants'].median())
df['percent-admittance'] = df['percent-admittance'].fillna(df['percent-admittance'].median())
df['percent-enrolled'] = df['percent-enrolled'].fillna(df['percent-enrolled'].median())
df['academics'] = df['academics'].fillna(df['academics'].median())
df['social'] = df['social'].fillna(df['social'].median())
df['quality-of-life'] = df['quality-of-life'].fillna(df['quality-of-life'].median())

df.head(4)

,university-name,state,location,control,number-of-students,male-percentage,student-percentage,sat-verbal,sat-math,expenses,percent-financial-aid,number-of-applicants,percent-admittance,percent-enrolled,academics,social,quality-of-life,academic-emphasis
0,ADELPHI,NEWYORK,SUBURBAN,PRIVATE,5000.0,0.30,0.937500,500.0,475.0,7000.0,0.6,4000.0,0.7,0.4,2.0,2.0,2.0,"['BUSINESS-ADMINISTRATION', 'BIOLOGY']"
1,ARIZONA-STATE,ARIZONA,URBAN,STATE,20000.0,0.50,0.952381,450.0,500.0,4000.0,0.5,17000.0,0.8,0.6,3.0,4.0,5.0,"['BUSINESS-EDUCATION', 'ENGINEERING', 'ACCOUNT..."
2,BOSTON-COLLEGE,MASSACHUSETTS,SUBURBAN,PRIVATE,5000.0,0.40,0.952381,500.0,550.0,10000.0,0.6,10000.0,0.5,0.4,4.0,5.0,3.0,"['ECONOMICS', 'BIOLOGY', 'ENGLISH']"
3,BOSTON-UNIVERSITY,MASSACHUSETTS,URBAN,PRIVATE,10000.0,0.45,0.923077,550.0,575.0,10000.0,0.6,13000.0,0.6,0.4,4.0,4.0,3.0,"['BUSINESS-ADMINISTRATION', 'PSYCHOLOGY', 'LIB..."


Finally, the data has been cleaned and we are ready to use it for our analysis.

## Exploratory Data Analysis